# Predicción del Precio del Oro

## Problema Real
¿Cuál será el precio del oro el próximo lunes? ¿Vale la pena invertir ahora?

## Objetivo
Predecir el precio del oro usando datos históricos REALES y técnicas de Machine Learning.

## Metodología
1. Obtener datos reales del precio del oro (últimos meses)
2. Análisis exploratorio de tendencias
3. Regresión lineal para predicción de precio
4. Predicción para el próximo lunes
5. Análisis de factores que afectan el precio

## Datos
- **Fuente**: Yahoo Finance (datos reales)
- **Símbolo**: GC=F (Gold Futures)
- **Período**: Últimos 6 meses hasta HOY

## 1. Instalación e Importación de Librerías

In [ ]:
# Instalar yfinance si no está disponible
# !pip install yfinance pandas numpy matplotlib seaborn scikit-learn

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Librerías importadas exitosamente")

## 2. Descarga de Datos Reales del Oro

In [ ]:
# Definir período de análisis (últimos 6 meses)
fecha_fin = datetime.now()
fecha_inicio = fecha_fin - timedelta(days=180)  # 6 meses

print("=" * 70)
print("📊 DESCARGANDO DATOS REALES DEL ORO")
print("=" * 70)
print(f"Período: {fecha_inicio.date()} a {fecha_fin.date()}")
print(f"Fuente: Yahoo Finance (GC=F - Gold Futures)")
print("\nDescargando...")

# Descargar datos del oro
oro = yf.download('GC=F', start=fecha_inicio, end=fecha_fin, progress=False)

print(f"\n✅ Datos descargados: {len(oro)} registros")
print(f"Fecha más reciente: {oro.index[-1].date()}")
print(f"Precio actual: ${oro['Close'].iloc[-1]:.2f} USD")

# Mostrar primeros registros
oro.head()

In [ ]:
# Información del dataset
print("=" * 70)
print("INFORMACIÓN DEL DATASET")
print("=" * 70)
print(oro.info())

print("\n" + "=" * 70)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 70)
print(oro.describe())

## 3. Exploración de Datos (EDA)

In [ ]:
# Visualizar evolución del precio del oro
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Gráfico 1: Precio de cierre
axes[0].plot(oro.index, oro['Close'], linewidth=2, color='gold', label='Precio de Cierre')
axes[0].fill_between(oro.index, oro['Close'], alpha=0.3, color='gold')
axes[0].set_ylabel('Precio (USD)', fontsize=12)
axes[0].set_title('📈 Evolución del Precio del Oro (Últimos 6 Meses)', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Volumen de transacciones
axes[1].bar(oro.index, oro['Volume'], color='steelblue', alpha=0.7)
axes[1].set_xlabel('Fecha', fontsize=12)
axes[1].set_ylabel('Volumen', fontsize=12)
axes[1].set_title('📊 Volumen de Transacciones', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Mostrar estadísticas clave
print("=" * 70)
print("📊 ESTADÍSTICAS CLAVE")
print("=" * 70)
print(f"Precio Mínimo: ${oro['Close'].min():.2f} USD")
print(f"Precio Máximo: ${oro['Close'].max():.2f} USD")
print(f"Precio Promedio: ${oro['Close'].mean():.2f} USD")
print(f"Precio Actual: ${oro['Close'].iloc[-1]:.2f} USD")
print(f"Volatilidad (Std): ${oro['Close'].std():.2f} USD")

In [ ]:
# Análisis de velas (candlestick simplificado)
fig, ax = plt.subplots(figsize=(15, 7))

# Últimos 60 días para mejor visualización
oro_reciente = oro.tail(60)

# Precio alto y bajo
ax.fill_between(oro_reciente.index, oro_reciente['Low'], oro_reciente['High'], 
                alpha=0.3, color='gray', label='Rango (Alto-Bajo)')
ax.plot(oro_reciente.index, oro_reciente['Close'], linewidth=2, color='gold', 
        marker='o', markersize=4, label='Precio de Cierre')
ax.plot(oro_reciente.index, oro_reciente['Open'], linewidth=1, color='blue', 
        alpha=0.5, linestyle='--', label='Precio de Apertura')

ax.set_xlabel('Fecha', fontsize=12)
ax.set_ylabel('Precio (USD)', fontsize=12)
ax.set_title('💰 Análisis de Rango de Precios (Últimos 60 días)', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Calcular retornos diarios
oro['Retorno_Diario'] = oro['Close'].pct_change() * 100
oro['Retorno_Acumulado'] = ((oro['Close'] / oro['Close'].iloc[0]) - 1) * 100

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico 1: Retornos diarios
axes[0].hist(oro['Retorno_Diario'].dropna(), bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Retorno Diario (%)', fontsize=11)
axes[0].set_ylabel('Frecuencia', fontsize=11)
axes[0].set_title('Distribución de Retornos Diarios', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Retorno acumulado
axes[1].plot(oro.index, oro['Retorno_Acumulado'], linewidth=2, color='green')
axes[1].fill_between(oro.index, oro['Retorno_Acumulado'], alpha=0.3, color='green')
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[1].set_xlabel('Fecha', fontsize=11)
axes[1].set_ylabel('Retorno Acumulado (%)', fontsize=11)
axes[1].set_title('Retorno Acumulado del Oro', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("=" * 70)
print("📈 ANÁLISIS DE RETORNOS")
print("=" * 70)
print(f"Retorno Total: {oro['Retorno_Acumulado'].iloc[-1]:.2f}%")
print(f"Retorno Promedio Diario: {oro['Retorno_Diario'].mean():.3f}%")
print(f"Volatilidad Diaria: {oro['Retorno_Diario'].std():.3f}%")
print(f"Mejor día: {oro['Retorno_Diario'].max():.2f}%")
print(f"Peor día: {oro['Retorno_Diario'].min():.2f}%")

## 4. Preparación de Datos para el Modelo

In [ ]:
# Crear características para el modelo
oro_modelo = oro.copy()

# Resetear índice para trabajar con números
oro_modelo.reset_index(inplace=True)
oro_modelo['Dia_Numero'] = range(len(oro_modelo))

# Características adicionales
oro_modelo['Dia_Semana'] = oro_modelo['Date'].dt.dayofweek
oro_modelo['Mes'] = oro_modelo['Date'].dt.month
oro_modelo['Precio_Anterior'] = oro_modelo['Close'].shift(1)
oro_modelo['Media_Movil_7'] = oro_modelo['Close'].rolling(window=7).mean()
oro_modelo['Media_Movil_30'] = oro_modelo['Close'].rolling(window=30).mean()
oro_modelo['Volatilidad_7'] = oro_modelo['Close'].rolling(window=7).std()

# Eliminar filas con NaN
oro_modelo = oro_modelo.dropna()

print("=" * 70)
print("CARACTERÍSTICAS CREADAS")
print("=" * 70)
print(oro_modelo[['Date', 'Close', 'Dia_Numero', 'Dia_Semana', 'Media_Movil_7', 'Media_Movil_30']].head(10))

print(f"\nTotal de registros después de limpieza: {len(oro_modelo)}")

In [ ]:
# Visualizar medias móviles
plt.figure(figsize=(15, 7))

plt.plot(oro_modelo['Date'], oro_modelo['Close'], linewidth=2, label='Precio Real', color='gold')
plt.plot(oro_modelo['Date'], oro_modelo['Media_Movil_7'], linewidth=2, label='Media Móvil 7 días', 
         color='blue', alpha=0.7)
plt.plot(oro_modelo['Date'], oro_modelo['Media_Movil_30'], linewidth=2, label='Media Móvil 30 días', 
         color='red', alpha=0.7)

plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Precio (USD)', fontsize=12)
plt.title('📊 Precio del Oro con Medias Móviles', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Modelo de Regresión Lineal

In [ ]:
# Preparar variables X e y
features = ['Dia_Numero', 'Dia_Semana', 'Mes', 'Precio_Anterior', 'Media_Movil_7', 
            'Media_Movil_30', 'Volatilidad_7', 'Volume']

X = oro_modelo[features]
y = oro_modelo['Close']

# Dividir en entrenamiento y prueba (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

print("=" * 70)
print("DIVISIÓN DE DATOS")
print("=" * 70)
print(f"Datos de entrenamiento: {len(X_train)} registros")
print(f"Datos de prueba: {len(X_test)} registros")
print(f"\nCaracterísticas utilizadas: {features}")

In [ ]:
# Entrenar modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Predicciones
y_pred_train = modelo.predict(X_train)
y_pred_test = modelo.predict(X_test)

# Evaluación del modelo
print("=" * 70)
print("⭐ EVALUACIÓN DEL MODELO")
print("=" * 70)

print(f"\n📊 Conjunto de Entrenamiento:")
print(f"  R² Score: {r2_score(y_train, y_pred_train):.4f}")
print(f"  RMSE: ${np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")
print(f"  MAE: ${mean_absolute_error(y_train, y_pred_train):.2f}")

print(f"\n📊 Conjunto de Prueba:")
print(f"  R² Score: {r2_score(y_test, y_pred_test):.4f}")
print(f"  RMSE: ${np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")
print(f"  MAE: ${mean_absolute_error(y_test, y_pred_test):.2f}")

r2_test = r2_score(y_test, y_pred_test)
if r2_test > 0.9:
    calidad = "EXCELENTE"
elif r2_test > 0.7:
    calidad = "BUENA"
elif r2_test > 0.5:
    calidad = "ACEPTABLE"
else:
    calidad = "MEJORABLE"

print(f"\n✅ Calidad del modelo: {calidad}")

In [ ]:
# Importancia de características
importancias = pd.DataFrame({
    'Característica': features,
    'Coeficiente': modelo.coef_
}).sort_values('Coeficiente', key=abs, ascending=False)

print("\n" + "=" * 70)
print("📈 IMPORTANCIA DE CARACTERÍSTICAS")
print("=" * 70)
print(importancias.to_string(index=False))

# Visualizar
plt.figure(figsize=(12, 6))
colors = ['green' if x > 0 else 'red' for x in importancias['Coeficiente']]
plt.barh(importancias['Característica'], importancias['Coeficiente'], color=colors, alpha=0.7)
plt.xlabel('Coeficiente', fontsize=12)
plt.title('Importancia de Características en el Modelo', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualizar predicciones vs reales
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Scatter plot
axes[0].scatter(y_test, y_pred_test, alpha=0.6, color='blue', edgecolors='k')
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', lw=2, label='Predicción Perfecta')
axes[0].set_xlabel('Precio Real (USD)', fontsize=12)
axes[0].set_ylabel('Precio Predicho (USD)', fontsize=12)
axes[0].set_title('Predicciones vs Valores Reales', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Serie temporal
indices_test = oro_modelo.iloc[X_test.index]['Date']
axes[1].plot(indices_test, y_test.values, linewidth=2, label='Precio Real', color='gold', marker='o')
axes[1].plot(indices_test, y_pred_test, linewidth=2, label='Precio Predicho', 
             color='blue', linestyle='--', marker='s')
axes[1].set_xlabel('Fecha', fontsize=12)
axes[1].set_ylabel('Precio (USD)', fontsize=12)
axes[1].set_title('Comparación Temporal de Predicciones', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. 🎯 PREDICCIÓN PARA EL PRÓXIMO LUNES

In [ ]:
# Calcular fecha del próximo lunes
hoy = datetime.now()
dias_hasta_lunes = (7 - hoy.weekday()) % 7
if dias_hasta_lunes == 0:
    dias_hasta_lunes = 7
proximo_lunes = hoy + timedelta(days=dias_hasta_lunes)

print("=" * 70)
print("🎯 PREDICCIÓN PARA EL PRÓXIMO LUNES")
print("=" * 70)
print(f"\nFecha de hoy: {hoy.strftime('%A %d/%m/%Y')}")
print(f"Próximo lunes: {proximo_lunes.strftime('%A %d/%m/%Y')}")
print(f"Días de diferencia: {dias_hasta_lunes}")

In [ ]:
# Preparar características para predicción del lunes
ultimo_registro = oro_modelo.iloc[-1]

# Características para el lunes
dia_numero_lunes = ultimo_registro['Dia_Numero'] + dias_hasta_lunes
dia_semana_lunes = 0  # Lunes = 0
mes_lunes = proximo_lunes.month
precio_anterior = ultimo_registro['Close']
media_movil_7 = oro_modelo.tail(7)['Close'].mean()
media_movil_30 = oro_modelo.tail(30)['Close'].mean()
volatilidad_7 = oro_modelo.tail(7)['Close'].std()
volumen_promedio = oro_modelo.tail(7)['Volume'].mean()

# Crear array de características
X_lunes = np.array([[
    dia_numero_lunes,
    dia_semana_lunes,
    mes_lunes,
    precio_anterior,
    media_movil_7,
    media_movil_30,
    volatilidad_7,
    volumen_promedio
]])

# Predecir precio del lunes
precio_predicho_lunes = modelo.predict(X_lunes)[0]

print("\n" + "=" * 70)
print("💰 RESULTADO DE LA PREDICCIÓN")
print("=" * 70)
print(f"\nPrecio actual del oro: ${precio_anterior:.2f} USD")
print(f"\n🔮 PRECIO PREDICHO PARA EL LUNES: ${precio_predicho_lunes:.2f} USD")

diferencia = precio_predicho_lunes - precio_anterior
porcentaje = (diferencia / precio_anterior) * 100

if diferencia > 0:
    print(f"\n📈 Tendencia: SUBIDA")
    print(f"   Incremento esperado: ${diferencia:.2f} ({porcentaje:+.2f}%)")
    print(f"   ✅ Recomendación: Podría ser buen momento para COMPRAR ahora")
else:
    print(f"\n📉 Tendencia: BAJADA")
    print(f"   Decremento esperado: ${diferencia:.2f} ({porcentaje:+.2f}%)")
    print(f"   ⚠️ Recomendación: Considerar ESPERAR o VENDER")

print("\n" + "=" * 70)

In [ ]:
# Crear predicciones para los próximos 7 días
predicciones_futuras = []
fechas_futuras = []

for i in range(1, 8):
    fecha_futura = hoy + timedelta(days=i)
    fechas_futuras.append(fecha_futura)
    
    # Solo predecir días laborables (lunes a viernes)
    if fecha_futura.weekday() < 5:
        dia_num = ultimo_registro['Dia_Numero'] + i
        dia_sem = fecha_futura.weekday()
        mes = fecha_futura.month
        
        # Usar última predicción como precio anterior para la siguiente
        if len(predicciones_futuras) > 0:
            precio_ant = predicciones_futuras[-1]
        else:
            precio_ant = precio_anterior
        
        X_futuro = np.array([[
            dia_num, dia_sem, mes, precio_ant,
            media_movil_7, media_movil_30, volatilidad_7, volumen_promedio
        ]])
        
        pred = modelo.predict(X_futuro)[0]
        predicciones_futuras.append(pred)
    else:
        predicciones_futuras.append(None)  # Fin de semana

# Visualizar predicciones futuras
plt.figure(figsize=(15, 7))

# Últimos 30 días históricos
oro_reciente = oro_modelo.tail(30)
plt.plot(oro_reciente['Date'], oro_reciente['Close'], linewidth=2.5, 
         label='Precio Histórico', color='gold', marker='o')

# Predicciones futuras
fechas_pred = [f for i, f in enumerate(fechas_futuras) if predicciones_futuras[i] is not None]
precios_pred = [p for p in predicciones_futuras if p is not None]

plt.plot(fechas_pred, precios_pred, linewidth=2.5, linestyle='--', 
         label='Predicción Próximos Días', color='red', marker='s', markersize=8)

# Marcar el lunes
plt.scatter([proximo_lunes], [precio_predicho_lunes], s=300, color='green', 
            marker='*', zorder=5, edgecolors='black', linewidth=2,
            label=f'Lunes: ${precio_predicho_lunes:.2f}')

plt.axvline(x=hoy, color='blue', linestyle=':', linewidth=2, alpha=0.7, label='Hoy')

plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Precio (USD)', fontsize=12)
plt.title('🔮 Predicción del Precio del Oro - Próximos Días', fontsize=14, fontweight='bold')
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 7. Análisis de Tendencias

In [ ]:
# Análisis de tendencia a corto, medio y largo plazo
print("=" * 70)
print("📊 ANÁLISIS DE TENDENCIAS")
print("=" * 70)

# Últimos 7 días
precio_7_dias_atras = oro_modelo.iloc[-7]['Close']
cambio_7_dias = ((precio_anterior - precio_7_dias_atras) / precio_7_dias_atras) * 100

# Últimos 30 días
precio_30_dias_atras = oro_modelo.iloc[-30]['Close']
cambio_30_dias = ((precio_anterior - precio_30_dias_atras) / precio_30_dias_atras) * 100

# Últimos 90 días
if len(oro_modelo) >= 90:
    precio_90_dias_atras = oro_modelo.iloc[-90]['Close']
    cambio_90_dias = ((precio_anterior - precio_90_dias_atras) / precio_90_dias_atras) * 100
else:
    precio_90_dias_atras = oro_modelo.iloc[0]['Close']
    cambio_90_dias = ((precio_anterior - precio_90_dias_atras) / precio_90_dias_atras) * 100

print(f"\n📅 Última semana (7 días):")
print(f"   Cambio: {cambio_7_dias:+.2f}%")
print(f"   Tendencia: {'📈 ALCISTA' if cambio_7_dias > 0 else '📉 BAJISTA'}")

print(f"\n📅 Último mes (30 días):")
print(f"   Cambio: {cambio_30_dias:+.2f}%")
print(f"   Tendencia: {'📈 ALCISTA' if cambio_30_dias > 0 else '📉 BAJISTA'}")

print(f"\n📅 Últimos 3 meses (90 días):")
print(f"   Cambio: {cambio_90_dias:+.2f}%")
print(f"   Tendencia: {'📈 ALCISTA' if cambio_90_dias > 0 else '📉 BAJISTA'}")

# Visualización
periodos = ['7 días', '30 días', '90 días']
cambios = [cambio_7_dias, cambio_30_dias, cambio_90_dias]
colors = ['green' if c > 0 else 'red' for c in cambios]

plt.figure(figsize=(10, 6))
plt.bar(periodos, cambios, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
plt.axhline(y=0, color='black', linestyle='-', linewidth=1)
plt.ylabel('Cambio (%)', fontsize=12)
plt.title('Cambio de Precio por Período', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

# Añadir valores en las barras
for i, v in enumerate(cambios):
    plt.text(i, v + (0.5 if v > 0 else -0.5), f'{v:+.2f}%', 
             ha='center', va='bottom' if v > 0 else 'top', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()

## 8. Factores que Afectan el Precio del Oro

In [ ]:
print("=" * 70)
print("🌍 FACTORES QUE AFECTAN EL PRECIO DEL ORO")
print("=" * 70)

print("""
1. 💵 INFLACIÓN
   - El oro es refugio contra inflación
   - Cuando sube inflación → sube oro

2. 💰 TASAS DE INTERÉS
   - Tasas altas → oro baja (bonos más atractivos)
   - Tasas bajas → oro sube

3. 💵 VALOR DEL DÓLAR
   - Dólar fuerte → oro baja
   - Dólar débil → oro sube

4. ⚠️ INCERTIDUMBRE GEOPOLÍTICA
   - Crisis/guerras → oro sube (activo refugio)
   - Estabilidad → oro baja

5. 📊 OFERTA Y DEMANDA
   - Demanda de joyería (India, China)
   - Compras de bancos centrales
   - Producción minera

6. 📈 MERCADOS BURSÁTILES
   - Bolsa cae → oro sube (refugio)
   - Bolsa sube → oro puede bajar
""")

print("=" * 70)

## 9. Dashboard Resumen

In [ ]:
# Dashboard completo
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Precio histórico (grande)
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(oro_modelo['Date'], oro_modelo['Close'], linewidth=2, color='gold')
ax1.fill_between(oro_modelo['Date'], oro_modelo['Close'], alpha=0.3, color='gold')
ax1.set_ylabel('Precio (USD)', fontsize=11)
ax1.set_title('💰 EVOLUCIÓN DEL PRECIO DEL ORO', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# 2. Predicción vs Real
ax2 = fig.add_subplot(gs[1, 0])
ax2.scatter(y_test, y_pred_test, alpha=0.6, color='blue')
ax2.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax2.set_xlabel('Real', fontsize=10)
ax2.set_ylabel('Predicho', fontsize=10)
ax2.set_title('Predicción vs Real', fontsize=11, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. Distribución de retornos
ax3 = fig.add_subplot(gs[1, 1])
ax3.hist(oro['Retorno_Diario'].dropna(), bins=30, color='steelblue', alpha=0.7, edgecolor='black')
ax3.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax3.set_xlabel('Retorno (%)', fontsize=10)
ax3.set_title('Distribución de Retornos', fontsize=11, fontweight='bold')
ax3.grid(True, alpha=0.3)

# 4. Tendencias
ax4 = fig.add_subplot(gs[1, 2])
periodos = ['7 días', '30 días', '90 días']
cambios_val = [cambio_7_dias, cambio_30_dias, cambio_90_dias]
colors_bar = ['green' if c > 0 else 'red' for c in cambios_val]
ax4.bar(periodos, cambios_val, color=colors_bar, alpha=0.7)
ax4.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax4.set_ylabel('Cambio (%)', fontsize=10)
ax4.set_title('Tendencias', fontsize=11, fontweight='bold')
ax4.grid(True, alpha=0.3, axis='y')

# 5. Predicción futura
ax5 = fig.add_subplot(gs[2, :2])
oro_recent = oro_modelo.tail(20)
ax5.plot(oro_recent['Date'], oro_recent['Close'], linewidth=2, color='gold', 
         marker='o', label='Histórico')
ax5.plot(fechas_pred, precios_pred, linewidth=2, linestyle='--', color='red', 
         marker='s', label='Predicción')
ax5.scatter([proximo_lunes], [precio_predicho_lunes], s=200, color='green', 
            marker='*', zorder=5, label='Lunes')
ax5.set_xlabel('Fecha', fontsize=10)
ax5.set_ylabel('Precio (USD)', fontsize=10)
ax5.set_title('Predicción Próximos Días', fontsize=11, fontweight='bold')
ax5.legend(fontsize=9)
ax5.grid(True, alpha=0.3)
ax5.tick_params(axis='x', rotation=45)

# 6. Métricas del modelo
ax6 = fig.add_subplot(gs[2, 2])
ax6.axis('off')
metricas_text = f"""
📊 MÉTRICAS DEL MODELO

R² Score: {r2_score(y_test, y_pred_test):.3f}
RMSE: ${np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}
MAE: ${mean_absolute_error(y_test, y_pred_test):.2f}

🎯 PREDICCIÓN LUNES
${precio_predicho_lunes:.2f}

Cambio: {porcentaje:+.2f}%
"""
ax6.text(0.1, 0.5, metricas_text, fontsize=11, verticalalignment='center',
         bbox=dict(boxstyle='round', facecolor='lightgoldenrodyellow', alpha=0.8))

plt.suptitle('📊 DASHBOARD: PREDICCIÓN DEL PRECIO DEL ORO', 
             fontsize=16, fontweight='bold', y=0.98)
plt.show()

## 10. 📋 Resumen y Recomendaciones

In [ ]:
print("=" * 70)
print("📋 RESUMEN EJECUTIVO")
print("=" * 70)

print(f"\n📊 DATOS ANALIZADOS:")
print(f"   - Período: {len(oro_modelo)} días de datos reales")
print(f"   - Fuente: Yahoo Finance (Gold Futures GC=F)")
print(f"   - Última actualización: {oro_modelo['Date'].iloc[-1].strftime('%d/%m/%Y')}")

print(f"\n💰 PRECIO ACTUAL:")
print(f"   - Precio: ${precio_anterior:.2f} USD")
print(f"   - Rango 6 meses: ${oro['Close'].min():.2f} - ${oro['Close'].max():.2f}")
print(f"   - Volatilidad: ${oro['Close'].std():.2f}")

print(f"\n🔮 PREDICCIÓN PARA EL {proximo_lunes.strftime('%d/%m/%Y')} (LUNES):")
print(f"   - Precio predicho: ${precio_predicho_lunes:.2f} USD")
print(f"   - Cambio esperado: {porcentaje:+.2f}% (${diferencia:+.2f})")

if diferencia > 0:
    print(f"   - Tendencia: 📈 ALCISTA")
    print(f"   - Señal: COMPRA")
    recomendacion = "COMPRAR"
else:
    print(f"   - Tendencia: 📉 BAJISTA")
    print(f"   - Señal: VENTA / ESPERA")
    recomendacion = "VENDER o ESPERAR"

print(f"\n📈 CALIDAD DEL MODELO:")
print(f"   - R² Score: {r2_score(y_test, y_pred_test):.4f} ({calidad})")
print(f"   - Error promedio: ${mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"   - Confiabilidad: {'Alta' if r2_test > 0.8 else 'Media' if r2_test > 0.6 else 'Baja'}")

print(f"\n💡 RECOMENDACIÓN FINAL:")
print(f"   ➡️  {recomendacion}")

if abs(porcentaje) < 0.5:
    print(f"   ⚠️  Cambio pequeño esperado - mercado estable")
elif abs(porcentaje) > 2:
    print(f"   ⚠️  Cambio significativo esperado - alta volatilidad")

print("\n📅 VERIFICACIÓN:")
print(f"   El lunes {proximo_lunes.strftime('%d/%m/%Y')} podrás comprobar:")
print(f"   - Si el precio real se acerca a ${precio_predicho_lunes:.2f}")
print(f"   - Si la tendencia ({('ALCISTA' if diferencia > 0 else 'BAJISTA')}) fue correcta")

print("\n" + "=" * 70)
print("\n✅ Análisis completado exitosamente!")
print("\nEste modelo usa datos REALES y Machine Learning para predecir")
print("el precio del oro. El lunes podrás verificar la precisión.")
print("\n" + "=" * 70)

## 11. 📝 Guardar Predicción para Verificación

In [ ]:
# Crear resumen para guardar
resumen_prediccion = {
    'fecha_prediccion': hoy.strftime('%Y-%m-%d %H:%M:%S'),
    'fecha_objetivo': proximo_lunes.strftime('%Y-%m-%d'),
    'precio_actual': precio_anterior,
    'precio_predicho': precio_predicho_lunes,
    'cambio_esperado_usd': diferencia,
    'cambio_esperado_porcentaje': porcentaje,
    'recomendacion': recomendacion,
    'r2_score': r2_score(y_test, y_pred_test),
    'mae': mean_absolute_error(y_test, y_pred_test)
}

# Convertir a DataFrame y mostrar
df_prediccion = pd.DataFrame([resumen_prediccion])

print("=" * 70)
print("📄 PREDICCIÓN GUARDADA PARA VERIFICACIÓN")
print("=" * 70)
print(df_prediccion.to_string(index=False))

# Opcionalmente guardar a CSV
# df_prediccion.to_csv('prediccion_oro_lunes.csv', index=False)
# print("\n✅ Predicción guardada en 'prediccion_oro_lunes.csv'")

print("\n💡 Guarda esta predicción y compárala el lunes con el precio real!")